#### Score: 26.0529

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.preprocessing import Imputer
import lightgbm as lgb

In [3]:
df_X_train = pd.read_csv('data_dengue/dengue_features_train.csv', index_col=None)
df_y_train = pd.read_csv('data_dengue/dengue_labels_train.csv', index_col=None)

In [4]:
X_train = df_X_train
X_train['week_start_date'] = pd.to_datetime(X_train['week_start_date'])
y_train = df_y_train['total_cases']
X_train['week_start_date'] = X_train['week_start_date'] - pd.to_datetime('1980-01-01')
X_train['week_start_date'] = pd.to_numeric(X_train['week_start_date'])/100000000000

In [6]:
X_train_sj = X_train[X_train.city=='sj']
X_train_iq = X_train[X_train.city=='iq']
y_train_sj = df_y_train[df_y_train.city=='sj']['total_cases']
y_train_iq = df_y_train[df_y_train.city=='iq']['total_cases']

In [7]:
X_train_sj = X_train_sj.drop(['city'], axis=1)
X_train_iq = X_train_iq.drop(['city'], axis=1)

In [8]:
X_train_sj = X_train_sj.fillna(X_train_sj.mean())
X_train_iq = X_train_iq.fillna(X_train_iq.mean())

In [9]:
X_test = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)

In [10]:
X_test['week_start_date'] = pd.to_datetime(X_test['week_start_date'])
X_test['week_start_date'] = X_test['week_start_date'] - pd.to_datetime('1980-01-01')
X_test['week_start_date'] = pd.to_numeric(X_test['week_start_date'])/100000000000
X_test_sj = X_test[X_test.city=='sj']
X_test_iq = X_test[X_test.city=='iq']
X_test_sj = X_test_sj.drop(['city'], axis=1)
X_test_iq = X_test_iq.drop(['city'], axis=1)
X_test_sj = X_test_sj.fillna(X_test_sj.mean())
X_test_iq = X_test_iq.fillna(X_test_sj.mean())

In [31]:
gbm_sj = lgb.LGBMRegressor(objective='regression')

In [32]:
param_grid = {
    'learning_rate': [0.01,0.1,1],
    'n_estimators': [10,20,40,70],
    'num_leaves': [5,10,30,50]
}

In [33]:
gbm_sj_grid = GridSearchCV(gbm_sj, param_grid, cv=4)
gbm_sj_grid.fit(X_train_sj, y_train_sj)

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective='regression', random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'n_estimators': [10, 20, 40, 70],
                         'num_leaves': [5, 10

In [34]:
print('Best parameters found by grid search are:', gbm_sj_grid.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 10, 'num_leaves': 10}


In [37]:
gbm_sj_best = lgb.LGBMRegressor(objective='regression',num_leaves=10, learning_rate=0.1, n_estimators=10)
gbm_sj_best.fit(X_train_sj, y_train_sj)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=10, n_jobs=-1, num_leaves=10, objective='regression',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [36]:
gbm_iq = lgb.LGBMRegressor(objective='regression')
gbm_iq_grid = GridSearchCV(gbm_iq, param_grid, cv=4)
gbm_iq_grid.fit(X_train_iq, y_train_iq)
print('Best parameters found by grid search are:', gbm_iq_grid.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.01, 'n_estimators': 10, 'num_leaves': 30}


In [38]:
%%time
gbm_iq_best = lgb.LGBMRegressor(objective='regression',num_leaves=30, learning_rate=0.01, n_estimators=10)
gbm_iq_best.fit(X_train_iq, y_train_iq)

Wall time: 38 ms


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.01, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=10, n_jobs=-1, num_leaves=30, objective='regression',
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [39]:
result_sj = gbm_sj_best.predict(X_test_sj)

In [40]:
result_columns_sj = X_test[X_test.city=='sj'][['city','year','weekofyear']]
result_cases_sj = pd.DataFrame(result_sj)
result_cases_sj.columns = ['total_cases']
result_df_sj = pd.concat([result_columns_sj, result_cases_sj], axis=1)
result_df_sj['total_cases'] = result_df_sj['total_cases'].round().astype(int)
result_df_sj.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,20
1,sj,2008,19,20
2,sj,2008,20,20
3,sj,2008,21,20
4,sj,2008,22,20


In [41]:
result_iq = gbm_iq_best.predict(X_test_iq)

In [42]:
result_columns_iq = X_test[X_test.city=='iq'][['city','year','weekofyear']].reset_index()
result_cases_iq = pd.DataFrame(result_iq)
result_cases_iq.columns = ['total_cases']
result_df_iq = pd.concat([result_columns_iq, result_cases_iq], axis=1)
result_df_iq['total_cases'] = result_df_iq['total_cases'].round().astype(int)
result_df_iq= result_df_iq.drop(['index'], axis=1)
result_df_iq.head()

,city,year,weekofyear,total_cases
0,iq,2010,26,7
1,iq,2010,27,7
2,iq,2010,28,7
3,iq,2010,29,7
4,iq,2010,30,7


In [43]:
result = pd.concat([result_df_sj,result_df_iq], axis=0)
result.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,20
1,sj,2008,19,20
2,sj,2008,20,20
3,sj,2008,21,20
4,sj,2008,22,20


In [44]:
result.to_csv('lightgbm.csv', index=False)